In [27]:
import numpy as np
import pandas as pd
import seaborn as sns
import pyodbc
import sqlite3
import matplotlib.pyplot as plt
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression

In [28]:
# slesConn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=./go_sales_schoon.accdb;')

import pyodbc
import sqlite3

def convert_accdb_to_sqlite(accdb_file, sqlite_file):
    # Connect to the Access database
    accdb_conn_str = r"Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=" + accdb_file
    accdb_conn = pyodbc.connect(accdb_conn_str)
    accdb_cursor = accdb_conn.cursor()

    # Get the table names from the Access database
    table_names = []
    for row in accdb_cursor.tables():
        if row.table_type == 'TABLE':
            table_names.append(row.table_name)

    # Connect to the SQLite database
    sqlite_conn = sqlite3.connect(sqlite_file)
    sqlite_cursor = sqlite_conn.cursor()

    # Iterate through each table and copy the data to SQLite
    for table_name in table_names:
        # Retrieve the table data from Access
        accdb_cursor.execute(f"SELECT * FROM [{table_name}]")
        table_data = accdb_cursor.fetchall()

        # Get the column names from Access
        accdb_cursor.execute(f"SELECT * FROM [{table_name}]")
        column_names = [column[0] for column in accdb_cursor.description]

        # Create the table in SQLite without specifying column data types
        create_table_query = f'CREATE TABLE "{table_name}" ' + "(" + ", ".join([f'"{col}"' for col in column_names]) + ")"
        sqlite_cursor.execute(create_table_query)

        # Generate placeholders based on the number of columns
        placeholders = ",".join(["?" for _ in column_names])

        # Insert the data into the SQLite table, converting column types if necessary
        sqlite_cursor.executemany(f'INSERT INTO "{table_name}" VALUES ({placeholders})', table_data)

    # Commit the changes and close the connections
    sqlite_conn.commit()
    sqlite_conn.close()
    accdb_conn.close()

    print("Conversion completed successfully.")



# Usage example
convert_accdb_to_sqlite("./go_staff_schoon.accdb;", "staff.sqlite")


Conversion completed successfully.
